In [51]:
%reset -f
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from linearmodels.iv import IV2SLS
import functions as fun
import statsmodels.formula.api as sm
import seaborn as sns
from matplotlib import pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [52]:
df = pd.read_csv('dataset.csv')
Nobs=df['ID'].count()
df['Intercept']=np.ones((Nobs,1))
df.rename(columns={'Market share':'Market_share'}, inplace=True)
df2 = df[df['Market_share'] != 0]

df2.head(20)

,ID,Year,Market_share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,Intercept
8,1,2021,0.010373,Aiways,U5,400,284621.700,201,34,SUV,C,CN,257,1.0
9,1,2022,0.005976,Aiways,U5,400,313681.829,201,34,SUV,C,CN,183,1.0
10,1,2023,0.002860,Aiways,U5,400,264524.000,201,34,SUV,C,CN,177,1.0
21,2,2023,0.000048,Aiways,U6,405,360638.000,214,34,SUV,C,CN,3,1.0
28,3,2019,0.040630,Audi,e-tron,375,979704.475,402,17,SUV,F,DE,222,1.0
29,3,2020,0.034680,Audi,e-tron,375,890101.410,402,17,SUV,F,DE,491,1.0
30,3,2021,0.010494,Audi,e-tron,375,800035.193,402,17,SUV,F,DE,260,1.0
31,3,2022,0.017570,Audi,e-tron,375,789723.656,402,17,SUV,F,DE,538,1.0
32,3,2023,0.001099,Audi,e-tron,375,673037.728,402,17,SUV,F,DE,68,1.0
41,4,2021,0.003391,Audi,e-tron GT,472,1278896.110,522,17,Sedan,F,DE,84,1.0


In [53]:
# Copy the dataframe
data = df2.copy().reset_index(drop=True)
data.head(20)

,ID,Year,Market_share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,Intercept
0,1,2021,0.010373,Aiways,U5,400,284621.700,201,34,SUV,C,CN,257,1.0
1,1,2022,0.005976,Aiways,U5,400,313681.829,201,34,SUV,C,CN,183,1.0
2,1,2023,0.002860,Aiways,U5,400,264524.000,201,34,SUV,C,CN,177,1.0
3,2,2023,0.000048,Aiways,U6,405,360638.000,214,34,SUV,C,CN,3,1.0
4,3,2019,0.040630,Audi,e-tron,375,979704.475,402,17,SUV,F,DE,222,1.0
5,3,2020,0.034680,Audi,e-tron,375,890101.410,402,17,SUV,F,DE,491,1.0
6,3,2021,0.010494,Audi,e-tron,375,800035.193,402,17,SUV,F,DE,260,1.0
7,3,2022,0.017570,Audi,e-tron,375,789723.656,402,17,SUV,F,DE,538,1.0
8,3,2023,0.001099,Audi,e-tron,375,673037.728,402,17,SUV,F,DE,68,1.0
9,4,2021,0.003391,Audi,e-tron GT,472,1278896.110,522,17,Sedan,F,DE,84,1.0


In [54]:
#Scale for better intepretation

data['Price'] = data['Price']/10_000 #(Change in ms(%) for change in pris in 10.000)
data['HP'] = data['HP']/10           #(Change in ms(%) for change in HP in 10)
data['Range'] = data['Range']/10     #(Change in ms(%) for change in rækkevidde in 10)

In [55]:
# Creating dummy for china
data['China'] = (data['Country'] == 'CN').astype(int)

# IV

In [56]:
fun.GH(data, 'Range', 0.5)
fun.GH(data, 'HP', 0.5)
fun.GH(data, 'Chargetime', 0.5)

,ID,Year,Market_share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,Intercept,China,Range_GH,HP_GH,Chargetime_GH
0,1,2021,0.010373,Aiways,U5,40.0,28.462170,20.1,34,SUV,C,CN,257,1.0,1,1499.9,740.5,1317
1,1,2022,0.005976,Aiways,U5,40.0,31.368183,20.1,34,SUV,C,CN,183,1.0,1,2153.5,1060.2,1796
2,1,2023,0.002860,Aiways,U5,40.0,26.452400,20.1,34,SUV,C,CN,177,1.0,1,2792.3,1501.0,2099
3,2,2023,0.000048,Aiways,U6,40.5,36.063800,21.4,34,SUV,C,CN,3,1.0,1,2746.8,1456.2,2099
4,3,2019,0.040630,Audi,e-tron,37.5,97.970447,40.2,17,SUV,F,DE,222,1.0,0,470.9,310.5,533
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,189,2023,0.000065,Volvo,EX30,47.5,36.824500,26.8,28,SUV,B,SE,4,1.0,0,2189.4,1777.6,1813
330,190,2021,0.014652,Volvo,XC40,45.7,46.206060,40.2,28,SUV,C,SE,363,1.0,0,1169.9,1026.5,1138
331,190,2022,0.033310,Volvo,XC40,45.7,41.626340,40.2,28,SUV,C,SE,1020,1.0,0,1702.8,1621.7,1483
332,190,2023,0.031752,Volvo,XC40,45.7,43.926660,40.2,28,SUV,C,SE,1965,1.0,0,2206.7,1978.6,1813


In [57]:
formula = 'np.log(Market_share) ~ 1 + [Price ~ Range_GH + HP_GH + Chargetime_GH] + Range + HP + Chargetime + China '
IV = IV2SLS.from_formula(formula, data).fit(cov_type='robust')
IV.summary#.tables[1]

<class 'linearmodels.compat.statsmodels.Summary'>
"""
                           IV-2SLS Estimation Summary                           
================================================================================
Dep. Variable:     np.log(Market_share)   R-squared:                     -1.1894
Estimator:                      IV-2SLS   Adj. R-squared:                -1.2228
No. Observations:                   334   F-statistic:                    73.809
Date:                  Sun, May 12 2024   P-value (F-stat)                0.0000
Time:                          14:14:29   Distribution:                  chi2(5)
Cov. Estimator:                  robust                                         
                                                                                
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept     -4.6767     1.4901    -3.1386     0.0017     -7.5972     -1.7562
Range          0.1395     0.0231     6.0487     0.0000      0.0943      0.1847
HP             0.1587     0.0585     2.7127     0.0067      0.0440      0.2734
Chargetime    -0.0478     0.0201    -2.3749     0.0176     -0.0872     -0.0083
China         -3.7943     0.6345    -5.9798     0.0000     -5.0380     -2.5507
Price         -0.1722     0.0448    -3.8463     0.0001     -0.2599     -0.0844
==============================================================================

Endogenous: Price
Instruments: Range_GH, HP_GH, Chargetime_GH
Robust Covariance (Heteroskedastic)
Debiased: False
"""

# Willingness to pay

In [58]:
alpha = IV.params[-1] # Price coefficient
beta = IV.params[:-1]
beta_alpha_ratio = [b / -alpha for b in beta[1:]] #Willingness to pay (excluding constant)

for i in range(len(IV.params.index[1:-1])): #[1:-1] to exclude constant and price
    print('W2P:', IV.params.index[1:-1][i], beta_alpha_ratio[i])

W2P: Range 0.8102751985243255
W2P: HP 0.9217635970210817
W2P: Chargetime -0.2773644277316249
W2P: China -22.036887734646204


NOTE: English delimiter\
8,102 DKK for every 10 increase in Range\
9,217 DKK for every 10 increase in HP\
-2,773 DKK for every 10 increase in Chargetime\
-220,368 DKK for Chinese cars OR -22,036 DKK for Chinese cars???

# Logit

In [59]:
# IMPORTANT: The data must be sorted by year and ID before running the function
#             Because the CCPs returned by the function are sorted by year and ID
logit_data = data.sort_values(['Year', 'ID']).reset_index(drop=True)
X = logit_data[['Intercept', 'Range', 'HP', 'Chargetime', 'China']]
p_j = logit_data['Price']
logit_data['CCP'] = fun.ccp(alpha, beta, data, X)

In [60]:
probability_ratio = fun.probability_ratio(logit_data, 2023)
probability_ratio

Model,U5,U6,e-tron,e-tron GT,Q4 e-tron,Q8 e-tron,I3,i4,i5,I7,...,ID.3,ID.4,ID.5,ID.7,ID.Buzz,up!,C40,EX30,XC40,Free
Model,,,,,,,,,,,,,,,,,,,,,
U5,1.0,3.970191,0.661227,309.46,1.099154,3.547225,0.191381,0.166703,0.64086,18611907.065377,...,0.056947,0.016675,0.012484,0.015862,1.383754,0.469459,0.004839,0.01222,0.006362,0.488975
U6,0.251877,1.0,0.166548,77.945872,0.276852,0.893465,0.048205,0.041989,0.161418,4687912.269659,...,0.014344,0.0042,0.003144,0.003995,0.348536,0.118246,0.001219,0.003078,0.001602,0.123162
e-tron,1.51234,6.004278,1.0,468.008664,1.662295,5.364609,0.289434,0.252111,0.969199,28147527.162204,...,0.086123,0.025218,0.018879,0.023988,2.092706,0.709982,0.007318,0.018481,0.009621,0.739496
e-tron GT,0.003231,0.012829,0.002137,1.0,0.003552,0.011463,0.000618,0.000539,0.002071,60143.17542,...,0.000184,0.000054,0.00004,0.000051,0.004472,0.001517,0.000016,0.000039,0.000021,0.00158
Q4 e-tron,0.90979,3.612042,0.601578,281.543726,1.0,3.227231,0.174117,0.151665,0.583049,16932933.671992,...,0.05181,0.015171,0.011357,0.014431,1.258926,0.427109,0.004402,0.011118,0.005788,0.444865
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
up!,2.130111,8.456949,1.408487,659.18427,2.341321,7.555984,0.407664,0.355095,1.365104,39645435.162622,...,0.121303,0.035519,0.026591,0.033787,2.947549,1.0,0.010307,0.02603,0.013552,1.041571
C40,206.66263,820.49011,136.650926,63953.817335,227.15412,733.078856,39.551383,34.451229,132.441875,3846385654.777198,...,11.768748,3.44605,2.579877,3.278012,285.970167,97.019635,1.0,2.525395,1.314776,101.05281
EX30,81.83377,324.895698,54.110705,25324.278572,89.947942,290.2828,15.661461,13.641915,52.444015,1523082528.524133,...,4.66016,1.364558,1.021574,1.298019,113.237778,38.417601,0.395978,1.0,0.520622,40.014648


In [61]:
logit_data['Model_year'] = logit_data['Model'] + '_' + logit_data['Year'].astype(str)
marginal_effects = fun.marginal_effects(logit_data, IV)
marginal_effects

,Intercept,Range,HP,Chargetime,China,Price
Model_year,,,,,,
I3_2013,-0.098474,0.002938,0.003342,-0.001006,-0.079895,-0.003625
Leaf_2013,-0.032786,0.000978,0.001113,-0.000335,-0.0266,-0.001207
Zoe_2013,-0.094815,0.002829,0.003218,-0.000968,-0.076926,-0.003491
Fortwo_2013,-0.000645,0.000019,0.000022,-0.000007,-0.000524,-0.000024
Model S_2013,-0.231587,0.006909,0.007859,-0.002365,-0.187892,-0.008526
...,...,...,...,...,...,...
up!_2023,-0.001102,0.000033,0.000037,-0.000011,-0.000894,-0.000041
C40_2023,-0.104451,0.003116,0.003545,-0.001067,-0.084744,-0.003846
EX30_2023,-0.041945,0.001251,0.001423,-0.000428,-0.034031,-0.001544


In [62]:
marginal_effects[IV.params.index].mean()

Intercept    -0.090564
Range         0.002702
HP            0.003073
Chargetime   -0.000925
China        -0.073477
Price        -0.003334
dtype: float64

In [63]:
elasticity = fun.elasticity(logit_data, IV)
elasticity

,Intercept,Range,HP,Chargetime,China,Price
Model_year,,,,,,
I3_2013,-4.576045,4.027078,2.593412,-0.841122,-0.0,-4.211875
Leaf_2013,-4.643666,4.543731,2.316558,-2.039039,-0.0,-4.337201
Zoe_2013,-4.579865,4.986816,2.082679,-2.619007,-0.0,-2.91958
Fortwo_2013,-4.676039,1.771575,1.269501,-2.865005,-0.0,-3.623162
Model S_2013,-0.244354,0.44393,0.559742,-0.074858,-0.0,-0.647342
...,...,...,...,...,...,...
up!_2023,-4.675583,3.570701,1.285244,-2.29178,-0.0,-3.009914
C40_2023,-4.56979,6.352724,6.234292,-1.306623,-0.0,-7.245711
EX30_2023,-4.634357,6.566907,4.214917,-1.325084,-0.0,-6.283065


In [64]:
elasticity[IV.params.index].mean()

Intercept    -4.522662
Range         5.275402
HP            3.780338
Chargetime   -1.581019
China        -0.442494
Price        -7.780701
dtype: float64

# Cost

In [65]:
logit_data['Cost']=fun.cost_original(logit_data, alpha)
logit_data

,ID,Year,Market_share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,...,Country,Sales,Intercept,China,Range_GH,HP_GH,Chargetime_GH,CCP,Model_year,Cost
0,10,2013,0.002237,BMW,I3,29.5,25.00000,16.7,18,Hatchback,...,DE,1,1.0,0,110.1,83.6,237,0.021519,I3_2013,16.998865
1,132,2013,0.472036,Nissan,Leaf,32.8,25.36900,14.7,43,Hatchback,...,JP,211,1.0,0,99.2,67.5,164,0.007060,Leaf_2013,0.981772
2,158,2013,0.205817,Renault,Zoe,36.5,17.31500,13.4,56,Hatchback,...,FR,92,1.0,0,128.7,67.5,139,0.020703,Zoe_2013,8.998184
3,167,2013,0.002237,Smart,Fortwo,12.7,21.04575,8.0,60,Hatchback,...,DE,1,1.0,0,185.3,84.2,139,0.000138,Fortwo_2013,-1226.729355
4,173,2013,0.250559,Tesla,Model S,60.9,71.95631,67.5,30,Liftback,...,US,112,1.0,0,137.1,60.9,225,0.947751,Model S_2013,71.774637
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,187,2023,0.001277,Volkswagen,up!,25.6,17.48530,8.1,48,Hatchback,...,DE,79,1.0,0,3737.4,2269.0,2733,0.000236,up!_2023,-713.362330
330,188,2023,0.008354,Volvo,C40,46.6,43.06649,40.2,28,SUV,...,SE,517,1.0,0,2212.4,1978.6,1813,0.022857,C40_2023,35.533503
331,189,2023,0.000065,Volvo,EX30,47.5,36.82450,26.8,28,SUV,...,SE,4,1.0,0,2189.4,1777.6,1813,0.009051,EX30_2023,17.800730
332,190,2023,0.031752,Volvo,XC40,45.7,43.92666,40.2,28,SUV,...,SE,1965,1.0,0,2206.7,1978.6,1813,0.017385,XC40_2023,34.022470


In [66]:
logit_data[logit_data['Cost']>logit_data['Price']]

,ID,Year,Market_share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,...,Country,Sales,Intercept,China,Range_GH,HP_GH,Chargetime_GH,CCP,Model_year,Cost


In [67]:
len(logit_data[logit_data['Cost']<0])

222